
Stabilities and accuracy in time

_As the wave speed used for calculating time step is the maximum wave speed at each time step, which would be constant for all grid points, we first use a constant $C^n$ to replace $u^n_j$ in the second term of the right hand side, without changing the stability_
$$u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{C^n \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$
_Then von Neumann stability analysis of the Lax method is written below_

$$\xi^{n+1}e^{ikj\Delta x} = \frac{1}{2}(\xi^n e^{ik(j+1)\Delta x} + \xi^n e^{ik(j-1)\Delta x}) - \frac{C^n \Delta t}{2 \Delta x}(\xi^ne^{ik(j+1)\Delta x} - \xi^ne^{ik(j-1)\Delta x})$$
$$ = \frac{1}{2}(\xi^n e^{ikj\Delta x}e^{ik\Delta x} + \xi^n e^{ikj\Delta x}/e^{ik\Delta x}) - \frac{C^n\Delta t}{2 \Delta x}(\xi^ne^{ikj\Delta x}e^{ik\Delta x} - \xi^ne^{ikj\Delta x}/e^{ik\Delta x})$$
$$ = \xi^n e^{ikj\Delta x}[\frac{1}{2}(e^{ik\Delta x} + 1/e^{ik\Delta x}) - \frac{C^n\Delta t}{2 \Delta x}(e^{ik\Delta x} - 1/e^{ik\Delta x})]$$
$$ = \xi^n e^{ikj\Delta x}[\cos({k\Delta x})-\frac{iC^n\Delta t}{\Delta x}\sin({k\Delta x})]$$

_where $\xi^{n}e^{ikj\Delta x}$ is one of the terms in the fourier expansion of the round-off error at step $n$ and grid point $j$. Given $A=\cos({k\Delta x})-\frac{iC^n\Delta t}{\Delta x}\sin({k\Delta x})$, we calculate (the square of) the absolute value of the complex number_ $A$
$$|A|^2= \cos^2({k\Delta x})+\left(\frac{C^n\Delta t}{\Delta x}\right)^2\sin^2({k\Delta x})$$
$$= 1-\sin^2({k\Delta x})+\left(\frac{C^n\Delta t}{\Delta x}\right)^2\sin^2({k\Delta x})$$
$$= 1-\left[1-\left(\frac{C^n\Delta t}{\Delta x}\right)^2\right]\sin^2({k\Delta x})$$
_Therefore, to have $|A|^2\le 1$, we need $\left[1-\left(\frac{C^n\Delta t}{\Delta x}\right)^2\right]\ge 0$, i.e., $\frac{C^n\Delta t}{\Delta x}\le 1$, where $C^n$ is in fact the allowed CFL number._

_From the results shown below, we find that at $t\approx 25$, the difference between the total computing times (but the same number of time steps and same CFL) of the Lax method and the backward differencing method is $<10^{-4}$ (due to the slight difference between the maximum speeds). However, the difference between the locations of the wave fronts of two numerical results is already $>0.2$, which means the difference between two propagation speeds is several orders of magnetide higher than the one expected while assuming the same propagation speed. This is due to the fact that the wave speed in front of the pulse is zero, and thus the du is always zero in front of the pulse._

In [1]:
import sys 
import os   
sys.path.append(os.getcwd()+'/..')
import numpy as np
import math
import matplotlib.pyplot as plt 
import nm_lib as nm
from typing import Type
from matplotlib import animation 
import matplotlib
matplotlib.use('TkAgg')

# --- Values reused in other tasks of ex_3 ---
x0 = -1.4 
xf = 2.0
xc = 0.7
W = 0.1
pi = math.pi
CFL = 0.98
A = 0.02
# ---------------------------

def tfunc_Burgers(xx, A, B):
    return A * (math.tanh((xx + xc) / W) - math.tanh((xx - xc) / W)) + B

def discretisation(x0, xf, nump, dt_float: Type[float]):
    nint = nump - 1
    dh = dt_float((xf - x0) / nint)
    xx = dt_float(np.arange(nump) * dh + x0) 
    xx_half = dt_float(np.arange(nump) * dh + x0 + dh * 0.5)
    return nint, dh, xx, xx_half

def run_Burgers(nump, nt, CFL, A, B, ddx1 = lambda x,y: nm.deriv_cent(x,y), ddx2 = lambda x,y: nm.deriv_bck(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_Burgers(xx[i], A, B)
 
    t1, unnt1 = nm.evolv_Lax_uadv_burgers(xx, hh, nt, CFL, ddx1, 'wrap', bnd_limits)
    t2, unnt2 = nm.evolv_uadv_burgers(xx, hh, nt, CFL, ddx2, 'wrap', bnd_limits)
        
    plt.figure()
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt1[:,0].min() - 0.1 * (unnt1[:,0].max() - unnt1[:,0].min()), unnt1[:,0].max() + 0.1 * (unnt1[:,0].max() - unnt1[:,0].min()))
    plt.xlabel("x")
    plt.ylabel("u") 
    line1, = plt.plot([], [], '--', label='Lax solution')
    line2, = plt.plot([], [], '--', label='Backward differencing solution') 

    plt.legend(loc='lower right') 
    time_text1 = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
    time_text2 = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text1.set_text('')
        time_text2.set_text('')
        return line1, line2
 
    def update(frame):
        line1.set_data(xx, unnt1[:,frame])  
        line2.set_data(xx, unnt2[:,frame]) 
        time_text1.set_text("Time1 = "+str(t1[frame]))
        time_text2.set_text("Time2 = "+str(t2[frame]))
        return line1, line2
 
    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t1.size-1, blit=True)
    plt.show(block=True)

nump = 65   
nt = 21
B = 0.0
run_Burgers(nump, nt, CFL, A, B)  

1- Diffusive

_First we complete the formula above by writing it as the combination of two different orders of derivatives_
$$(u^{n+1}_j - u^{n}_j) / {\Delta t} = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) / {\Delta t} - (u^{n}_j) / {\Delta t} - \frac{u^n_{j}}{2 \Delta x}(u^n_{j+1} - u^n_{j-1}) $$
$$ = \frac{1}{2}(u^n_{j+1} - 2u^n_{j} + u^n_{j-1}) / {\Delta t} - \frac{u^n_{j}}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$
$$ = \frac{1}{2}[(u^n_{j+1} - u^n_{j}) - (u^n_{j} - u^n_{j-1})] / {\Delta t} - \frac{u^n_{j}}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$
$$ = \frac{\Delta^2 x}{2 \Delta^2 x\Delta t}[(u^n_{j+1} - u^n_{j}) - (u^n_{j} - u^n_{j-1})] - \frac{u^n_{j}}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$
_So for the whole computational domain, where the maximum wave speed is used to calculate the time step, we have_
$$u^{n+1}=u^{n} - \frac{\Delta t}{2\Delta x}u^n_{j}(u^n_{j+1} - u^n_{j-1})+ \frac{{\Delta^2 t}}{2{\Delta^2 x}} {\max}^2(u^n)[(u^n_{j+1} - u^n_{j}) - (u^n_{j} - u^n_{j-1})]$$
_The formula above can be rewritten as_ 
$$u^{n+1}=u^{n} - \frac{\Delta t}{2}u^n_{j}\left(\frac{\partial u}{\partial x}\right)_j+\frac{{\Delta^2 t}}{2} {\max}^2(u^n)\left(\frac{\partial^2 u}{\partial x^2}\right)_j$$
_As the second term on the right hand side is actually a diffusive term, it leads to strong diffusion when the __maximum speed__ is large. Moreover, the ratio between the second and first order derivatives are related also related to the time step._

_The results below show that the Lax method is more diffusive than the backward diferencing method, especially when the CFL is relatively small. However, the test below also shows that the Lax method should be second-order accurate, which means that its error decreases faster than the backward difference when finer meshes are used._

In [ ]:
#nt = 100
#CFL=0.98
#B = 0.0
#nump = 257
#run_Burgers(nump, nt, CFL, A, B) 
#nt = 200
#CFL=0.98
#B = 0.3
#nump = 257
#run_Burgers(nump, nt, CFL, A, B)    
#nt = 400
#CFL=0.49
#B = 0.3
#nump = 257
#run_Burgers(nump, nt, CFL, A, B) 
#nt = 800
#CFL=0.49
#B = 0.3
#nump = 513
#run_Burgers(nump, nt, CFL, A, B) 

x0 = -2.6 
xf = 2.6 

def tfunc(xx):
    return math.cos(6.0 * pi * xx / 5.0) * math.cos(6.0 * pi * xx / 5.0) /  math.cosh(5.0 * xx * xx)

def order_Burgers(nt, CFL, ddx1 = lambda x,y: nm.deriv_cent(x,y), ddx2 = lambda x,y: nm.deriv_bck(x,y), bnd_limits: list=[1,1],):
    nump1 = 65
    nint1, dh1, xx1, xx_half1 = discretisation(x0, xf, nump1, dt_float=np.float64)

    hh1 = np.zeros(nump1)
    for i in range(0, nump1):
        hh1[i] = tfunc(xx1[i])
 
    t11, unnt11 = nm.evolv_Lax_uadv_burgers(xx1, hh1, nt, CFL, ddx1, 'wrap', bnd_limits)
    t12, unnt12 = nm.evolv_uadv_burgers(xx1, hh1, nt, CFL, ddx2, 'wrap', bnd_limits)

    nump2 = 129
    nint2, dh2, xx2, xx_half2 = discretisation(x0, xf, nump2, dt_float=np.float64)

    hh2 = np.zeros(nump2)
    for i in range(0, nump2):
        hh2[i] = tfunc(xx2[i])
 
    t21, unnt21 = nm.evolv_Lax_uadv_burgers(xx2, hh2, nt*2, CFL, ddx1, 'wrap', bnd_limits)
    t22, unnt22 = nm.evolv_uadv_burgers(xx2, hh2, nt*2, CFL, ddx2, 'wrap', bnd_limits)

    nump3 = 257
    nint3, dh3, xx3, xx_half3 = discretisation(x0, xf, nump3, dt_float=np.float64)

    hh3 = np.zeros(nump3)
    for i in range(0, nump3):
        hh3[i] = tfunc(xx3[i])
 
    t31, unnt31 = nm.evolv_Lax_uadv_burgers(xx3, hh3, nt*4, CFL, ddx1, 'wrap', bnd_limits)
    t32, unnt32 = nm.evolv_uadv_burgers(xx3, hh3, nt*4, CFL, ddx2, 'wrap', bnd_limits)

    print("The order of accuracy of the Lax method is", nm.order_conv(unnt11[:,nt-1], unnt21[:,nt*2-1], unnt31[:,nt*4-1]))
    print("The order of accuracy of the backward difference is", nm.order_conv(unnt12[:,nt-1], unnt22[:,nt*2-1], unnt32[:,nt*4-1]))

nt = 15
CFL = 0.9 
order_Burgers(nt, CFL)

The order of accuracy of the Lax method is 2.293374537005224
The order of accuracy of the backward difference is 0.9795337285066997


: 